In [2]:
import pandas as pd
import utils

In [3]:
# Read the csv file and preprocess it: convert qualitative attributes to integers
train = pd.read_csv("train.csv")
# drop duplicates
train = train.drop_duplicates()
X = train.agg(utils.transform_dataset) # utils.transform_dataset is a dicitionary which applies a transforming function on each column
y= X["Body_Level"]
y= y-1
X = X.drop("Body_Level", axis=1)
X["BMI"] = X["Weight"].astype(float) / (X["Height"] ** 2).astype(float)


# Split the dataset into train and test sets
# sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

# test to test and val
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=.2, random_state=42)

from imblearn.over_sampling import SMOTE

# Initialize the LazyClassifier and fit the training data
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self, dropout=0):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(17, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 15)
        self.fc5 = nn.Linear(15, 4)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # x= self.dropout(x)
        x = torch.relu(self.fc2(x))
        x= self.dropout(x)
        x = torch.relu(self.fc3(x))
        # x= self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        # x = torch.softmax(x, dim=1)  # Apply the softmax function along the class dimension

        return x



In [5]:

X_train_tensor = torch.from_numpy(X_train.values).float()
X_test_tensor = torch.from_numpy(X_test.values).float()
X_val_tensor = torch.from_numpy(X_val.values).float()

y_train_tensor = torch.from_numpy(y_train.values).long()
y_test_tensor = torch.from_numpy(y_test.values).long()
y_val_tensor = torch.from_numpy(y_val.values).long()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

net = Net(dropout=0.2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=0.001)

# Train the neural network with L2 regularization
count =0
model_fin = None
last_val_acc = 0
for epoch in range(500):
    running_loss = 0.0
    count +=1
    for i, data in enumerate(train_loader, 0):
        # Get the inputs
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    if count % 50 == 0:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0
        # print acc
        with torch.no_grad():
            outputs = net(X_train_tensor)
            _, predicted_train = torch.max(outputs.data, 1)
        print('Accuracy of the network on the train data: %.3f' % (100 * torch.sum(y_train_tensor == predicted_train) / len(y_train_tensor)))
        # get val
        with torch.no_grad():
            outputs = net(X_val_tensor)
            _, predicted_val = torch.max(outputs.data, 1)
        val_acc= 100 * torch.sum(y_val_tensor == predicted_val) / len(y_val_tensor)
        if (val_acc >= last_val_acc) :
            last_val_acc = val_acc
            model_fin = net
        print('Accuracy of the network on the val data: %.3f' % (100 * torch.sum(y_val_tensor == predicted_val) / len(y_val_tensor)))

[50,    68] loss: 0.004
Accuracy of the network on the train data: 93.148
Accuracy of the network on the val data: 98.305
[100,    68] loss: 0.003
Accuracy of the network on the train data: 93.657
Accuracy of the network on the val data: 94.915
[150,    68] loss: 0.001
Accuracy of the network on the train data: 97.407
Accuracy of the network on the val data: 100.000
[200,    68] loss: 0.002
Accuracy of the network on the train data: 95.417
Accuracy of the network on the val data: 96.610
[250,    68] loss: 0.002
Accuracy of the network on the train data: 87.361
Accuracy of the network on the val data: 93.220
[300,    68] loss: 0.002
Accuracy of the network on the train data: 99.352
Accuracy of the network on the val data: 96.610
[350,    68] loss: 0.002
Accuracy of the network on the train data: 99.306
Accuracy of the network on the val data: 98.305
[400,    68] loss: 0.001
Accuracy of the network on the train data: 99.120
Accuracy of the network on the val data: 96.610
[450,    68] los

In [7]:
from sklearn.metrics import f1_score
#get acc test
with torch.no_grad():
    outputs = model_fin(X_test_tensor)
    _, predicted_test = torch.max(outputs.data, 1)
# print acc
print('Accuracy of the network on the test data: %.3f' % (100 * torch.sum(y_test_tensor == predicted_test) / len(y_test_tensor)))
# get f1
print('F1 score of the network on the test data: %.3f' % (f1_score(y_test_tensor, predicted_test, average='macro')))


# get acc train
with torch.no_grad():
    outputs = model_fin(X_train_tensor)
    _, predicted_train = torch.max(outputs.data, 1)
# print acc
print('Accuracy of the network on the train data: %.3f' % (100 * torch.sum(y_train_tensor == predicted_train) / len(y_train_tensor)))

Accuracy of the network on the test data: 97.863
F1 score of the network on the test data: 0.979
Accuracy of the network on the train data: 98.981


In [8]:
model_fin.eval()

Net(
  (fc1): Linear(in_features=17, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=15, bias=True)
  (fc5): Linear(in_features=15, out_features=4, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [9]:
# save model
torch.save(model_fin.state_dict(), "ffn_model.pt")